<a href="https://colab.research.google.com/github/praseedm/tracin_implemention_on_ledgar_dataset/blob/main/Train_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone git repo for data
! git clone https://github.com/praseedm/tracin_implemention_on_ledgar_dataset.git

Cloning into 'tracin_implemention_on_ledgar_dataset'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 20 (delta 5), reused 11 (delta 3), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [2]:
%cd tracin_implemention_on_ledgar_dataset

/content/tracin_implemention_on_ledgar_dataset


In [6]:
import os

import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import pandas as pd


tf.get_logger().setLevel('ERROR')

In [4]:
tf.__version__

'2.8.0'

## Data preparation

In [15]:
BATCH_SIZE = 32
SEED = 10
AUTOTUNE = tf.data.AUTOTUNE

In [10]:
train_data_path = './data/train_data.csv'
train_df = pd.read_csv(train_data_path)
print(train_df.shape)
train_df.head(3)

(32906, 2)


,text,label
0,Except as otherwise set forth in this Debentur...,97
1,This Amendment may be executed by one or more ...,26
2,"From time to time, as and when required by the...",45


In [27]:
def generate_batch_dataset_from_csv(csv_path:str):
  df = pd.read_csv(csv_path)
  dataset = tf.data.Dataset.from_tensor_slices({'text': df['text'], "label" : df['label']})
  print(f"{len(dataset)}\n{dataset.element_spec}")
  return dataset.batch(batch_size=BATCH_SIZE)

In [31]:
train_ds = tf.data.Dataset.from_tensor_slices({'text': train_df['text'], "label" : train_df['label']})

In [18]:
len(train_ds)

32906

In [20]:
train_ds.element_spec

{'label': TensorSpec(shape=(), dtype=tf.int64, name=None),
 'text': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [39]:
for row in train_ds.batch(3).take(1):
  print(row)

{'text': <tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'Except as otherwise set forth in this Debenture, the Company, for itself and its legal representatives, successors and assigns, expressly waives presentment, protest, demand, notice of dishonor, notice of nonpayment, notice of maturity, notice of protest, presentment for the purpose of accelerating maturity, and diligence in collection.',
       b'This Amendment may be executed by one or more of the parties hereto on any number of separate counterparts, and all of said counterparts taken together shall be deemed to constitute one and the same instrument. This Amendment may be delivered by facsimile or other electronic transmission of the relevant signature pages hereof.',
       b'From time to time, as and when required by the Surviving Corporation or by its successors or assigns, there shall be executed and delivered on behalf of Ashford (DE) such deeds and other instruments, and there shall be taken or caused to be taken 

## BERT Config
[BERT TF Hub](https://tfhub.dev/google/collections/bert/1)

In [5]:
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

In [ ]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)